# 환경설치

In [1]:
# 필요한 라이브러리 설치 (코랩에서 실행)
!pip install torch torchvision matplotlib seaborn pandas numpy Pillow scikit-learn

# 기본 라이브러리 import
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import time
import os

# 시드 설정 (재현 가능한 결과를 위해)
torch.manual_seed(42)
np.random.seed(42)

print(" 모든 라이브러리가 성공적으로 로드되었습니다!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/kwangsiklee/miniforge3/envs/llmenv/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can igno

 모든 라이브러리가 성공적으로 로드되었습니다!
PyTorch 버전: 2.5.1
CUDA 사용 가능: False


메모리 효율적인 데이터셋

In [2]:
class MemoryEfficientDataset(Dataset):
    """메모리 효율적인 데이터셋 (지연 로딩)"""

    def __init__(self, data_size=10000, feature_dim=100, cache_size=1000):
        """
        Args:
            data_size: 전체 데이터 크기
            feature_dim: 특성 차원
            cache_size: 캐시할 데이터 수
        """
        self.data_size = data_size
        self.feature_dim = feature_dim
        self.cache_size = cache_size

        # 데이터를 실제로 생성하지 않고 메타데이터만 저장
        self.labels = torch.randint(0, 10, (data_size,))

        # LRU 캐시 구현
        self.cache = {}
        self.access_order = []

        print(f" 메모리 효율적 데이터셋 생성:")
        print(f"   - 데이터 크기: {data_size:,}")
        print(f"   - 특성 차원: {feature_dim}")
        print(f"   - 캐시 크기: {cache_size}")
        print(f"   - 메모리 사용량: 라벨만 저장 (~{data_size * 4 / 1024**2:.1f}MB)")


    def _generate_data(self, idx):
        """데이터 생성 (실제로는 파일에서 로드)"""
        # 시뮬레이션: 실제로는 파일 I/O나 복잡한 계산
        torch.manual_seed(idx)  # 일관된 데이터 생성을 위해
        data = torch.randn(self.feature_dim)
        time.sleep(0.001)  # I/O 지연 시뮬레이션
        return data

    def _update_cache(self, idx, data):
        """LRU 캐시 업데이트"""
        # 캐시 크기 초과 시 가장 오래된 데이터 제거
        if len(self.cache) >= self.cache_size:
            oldest_idx = self.access_order.pop(0)
            del self.cache[oldest_idx]

        # 새 데이터 캐시에 추가
        self.cache[idx] = data
        self.access_order.append(idx)

    def __len__(self):
        return self.data_size

    def __getitem__(self, idx):
        # 캐시 확인
        if idx in self.cache:
            # 캐시 히트: 접근 순서 업데이트
            self.access_order.remove(idx)
            self.access_order.append(idx)
            data = self.cache[idx]
        else:
            # 캐시 미스: 데이터 생성 및 캐시 저장
            data = self._generate_data(idx)
            self._update_cache(idx, data)

        label = self.labels[idx]
        return data, label

    def get_cache_stats(self):
        """캐시 통계 반환"""
        return {
            'cache_size': len(self.cache),
            'max_cache_size': self.cache_size,
            'cache_utilization': len(self.cache) / self.cache_size * 100
        }

# 메모리 효율성 테스트
print(" 메모리 효율성 테스트:")

# 일반 데이터셋 vs 메모리 효율적 데이터셋 비교
print("\n1 일반 데이터셋 (모든 데이터 메모리에 로드):")
start_time = time.time()
# normal_dataset = NumberDataset(num_samples=1000, input_dim=100)
normal_dataset = MemoryEfficientDataset(data_size=1000, feature_dim=100, cache_size=1000) # Using MemoryEfficientDataset for comparison
normal_creation_time = time.time() - start_time
print(f"   생성 시간: {normal_creation_time:.3f}초")

print("\n2 메모리 효율적 데이터셋 (지연 로딩):")
start_time = time.time()
efficient_dataset = MemoryEfficientDataset(data_size=1000, feature_dim=100, cache_size=100)
efficient_creation_time = time.time() - start_time
print(f"   생성 시간: {efficient_creation_time:.3f}초")

# 캐시 효과 테스트
print(f"\n 캐시 효과 테스트:")

# 첫 번째 접근 (캐시 미스)
start_time = time.time()
for i in range(50):
    _ = efficient_dataset[i]
first_access_time = time.time() - start_time

# 두 번째 접근 (캐시 히트)
start_time = time.time()
for i in range(50):
    _ = efficient_dataset[i]
second_access_time = time.time() - start_time

print(f"   첫 번째 접근 (캐시 미스): {first_access_time:.3f}초")
print(f"   두 번째 접근 (캐시 히트): {second_access_time:.3f}초")
print(f"   속도 향상: {first_access_time / second_access_time:.1f}x")

# 캐시 통계
cache_stats = efficient_dataset.get_cache_stats()
print(f"\n 캐시 통계:")
for key, value in cache_stats.items():
    if 'utilization' in key:
        print(f"   {key}: {value:.1f}%")
    else:
        print(f"   {key}: {value}")


 메모리 효율성 테스트:

1 일반 데이터셋 (모든 데이터 메모리에 로드):
 메모리 효율적 데이터셋 생성:
   - 데이터 크기: 1,000
   - 특성 차원: 100
   - 캐시 크기: 1000
   - 메모리 사용량: 라벨만 저장 (~0.0MB)
   생성 시간: 0.007초

2 메모리 효율적 데이터셋 (지연 로딩):
 메모리 효율적 데이터셋 생성:
   - 데이터 크기: 1,000
   - 특성 차원: 100
   - 캐시 크기: 100
   - 메모리 사용량: 라벨만 저장 (~0.0MB)
   생성 시간: 0.000초

 캐시 효과 테스트:
   첫 번째 접근 (캐시 미스): 0.085초
   두 번째 접근 (캐시 히트): 0.000초
   속도 향상: 612.5x

 캐시 통계:
   cache_size: 50
   max_cache_size: 100
   cache_utilization: 50.0%


# 성능 최적화

In [3]:
import time

def benchmark_dataloader_settings():
    """DataLoader 설정별 성능 벤치마크"""

    print(" DataLoader 성능 벤치마크")
    print("=" * 50)

    # 테스트용 큰 데이터셋 생성
    # large_dataset = NumberDataset(num_samples=5000, input_dim=100, num_classes=10)
    # Use MemoryEfficientDataset instead of NumberDataset
    large_dataset = MemoryEfficientDataset(data_size=5000, feature_dim=100, cache_size=500)


    # 다양한 설정 테스트
    test_configs = [
        {'name': '기본설정', 'batch_size': 32, 'num_workers': 0, 'pin_memory': False},
        {'name': '큰배치', 'batch_size': 128, 'num_workers': 0, 'pin_memory': False},
        {'name': '멀티워커', 'batch_size': 32, 'num_workers': 2, 'pin_memory': False},
        {'name': '핀메모리', 'batch_size': 32, 'num_workers': 0, 'pin_memory': True},
        {'name': '최적화', 'batch_size': 64, 'num_workers': 2, 'pin_memory': True}
    ]

    results = []

    for config in test_configs:
        print(f"\n 테스트 중: {config['name']}")

        # DataLoader 생성
        dataloader = DataLoader(
            large_dataset,
            batch_size=config['batch_size'],
            shuffle=True,
            num_workers=config['num_workers'],
            pin_memory=config['pin_memory'] and torch.cuda.is_available()
        )

        # 시간 측정
        start_time = time.time()
        batch_count = 0

        for batch_data, batch_labels in dataloader:
            # 간단한 연산 시뮬레이션
            _ = batch_data.mean()
            _ = batch_labels.sum()

            batch_count += 1
            if batch_count >= 20:  # 20개 배치만 테스트
                break

        elapsed_time = time.time() - start_time
        throughput = (batch_count * config['batch_size']) / elapsed_time

        result = {
            'name': config['name'],
            'time': elapsed_time,
            'throughput': throughput,
            'config': config
        }
        results.append(result)

        print(f"   ⏱ 시간: {elapsed_time:.3f}초")
        print(f"    처리량: {throughput:.1f} 샘플/초")

    # 결과 비교
    print(f"\n 성능 비교 결과:")
    print("-" * 60)
    baseline = results[0]['throughput']

    for result in results:
        speedup = result['throughput'] / baseline
        print(f"{result['name']:10} | {result['time']:6.3f}초 | {result['throughput']:8.1f} 샘플/초 | {speedup:4.2f}x")

    return results

# 성능 벤치마크 실행
benchmark_results = benchmark_dataloader_settings()

 DataLoader 성능 벤치마크
 메모리 효율적 데이터셋 생성:
   - 데이터 크기: 5,000
   - 특성 차원: 100
   - 캐시 크기: 500
   - 메모리 사용량: 라벨만 저장 (~0.0MB)

 테스트 중: 기본설정
   ⏱ 시간: 1.099초
    처리량: 582.2 샘플/초

 테스트 중: 큰배치
   ⏱ 시간: 4.544초
    처리량: 563.3 샘플/초

 테스트 중: 멀티워커


Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
  File "<string>", line 1, in <module>
  File "/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
               ^^^^^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^^^
^^^^  File "/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
^^^^^^^^^^^^^^^
  File "/Users/kwangsiklee/miniforge3/envs/llmenv/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
      self = reduction.pickle.load(from_parent) 
        ^^^^^^^^^^^^^^^^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^^^^^
^^^^^AttributeError^^: ^Can't get attribute 'MemoryEfficientDataset' on <

RuntimeError: DataLoader worker (pid(s) 80008, 80009) exited unexpectedly

GPU 최적화

In [9]:
class GPUOptimizedLoader:
    """GPU 최적화된 데이터 로더"""

    def __init__(self, dataset, batch_size=32, shuffle=True):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=2 if not torch.cuda.is_available() else 0,  # GPU 사용시 워커 줄임
            pin_memory=torch.cuda.is_available(),  # GPU 사용시 pin memory 활성화
            persistent_workers=True if not torch.cuda.is_available() else False
        )

        print(f" GPU 최적화 로더 생성:")
        print(f"   - Device: {self.device}")
        print(f"   - Pin memory: {torch.cuda.is_available()}")
        print(f"   - Workers: {self.dataloader.num_workers}")

    def __iter__(self):
        for batch_data, batch_labels in self.dataloader:
            # GPU로 비동기 전송
            batch_data = batch_data.to(self.device, non_blocking=True)
            batch_labels = batch_labels.to(self.device, non_blocking=True)

            yield batch_data, batch_labels

    def __len__(self):
        return len(self.dataloader)

# GPU 최적화 테스트
if torch.cuda.is_available():
    print(" GPU 최적화 테스트:")

    test_dataset = MemoryEfficientDataset(data_size=1000, feature_dim=50)

    # 일반 로더
    normal_loader = DataLoader(test_dataset, batch_size=32)

    # GPU 최적화 로더
    gpu_loader = GPUOptimizedLoader(test_dataset, batch_size=32)

    # 성능 비교
    print("\n⏱ 성능 비교:")

    # 일반 로더 테스트
    start_time = time.time()
    for i, (data, labels) in enumerate(normal_loader):
        data = data.cuda()
        labels = labels.cuda()
        _ = data.mean()  # 간단한 연산
        if i >= 10:
            break
    normal_time = time.time() - start_time

    # GPU 최적화 로더 테스트
    start_time = time.time()
    for i, (data, labels) in enumerate(gpu_loader):
        _ = data.mean()  # 간단한 연산
        if i >= 10:
            break
    optimized_time = time.time() - start_time

    print(f"   일반 로더: {normal_time:.3f}초")
    print(f"   GPU 최적화: {optimized_time:.3f}초")
    print(f"   성능 향상: {normal_time / optimized_time:.2f}x")

else:
    print(" GPU를 사용할 수 없어 CPU에서 테스트합니다.")
    print("   코랩에서 GPU 런타임을 선택하면 GPU 최적화를 테스트할 수 있습니다.")

# 메모리 사용량 모니터링 (GPU가 있는 경우)
if torch.cuda.is_available():
    print(f"\n GPU 메모리 사용량:")
    print(f"   할당된 메모리: {torch.cuda.memory_allocated() / 1024**2:.1f}MB")
    print(f"   캐시된 메모리: {torch.cuda.memory_reserved() / 1024**2:.1f}MB")

 GPU 최적화 테스트:
 메모리 효율적 데이터셋 생성:
   - 데이터 크기: 1,000
   - 특성 차원: 50
   - 캐시 크기: 1000
   - 메모리 사용량: 라벨만 저장 (~0.0MB)
 GPU 최적화 로더 생성:
   - Device: cuda
   - Pin memory: True
   - Workers: 0

⏱️ 성능 비교:
   일반 로더: 0.519초
   GPU 최적화: 0.337초
   성능 향상: 1.54x

 GPU 메모리 사용량:
   할당된 메모리: 0.0MB
   캐시된 메모리: 2.0MB
